## Домашнее задание 7. Обучение без учителя

В этом задании мы рассмотрим, как работают методы снижения размерности и кластеризации. Заодно ещё раз попрактикуемся в задаче классификации.

Мы будем работать с датасетом [Samsung Human Activity Recognition](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones). Скачайте данные [здесь](https://drive.google.com/file/d/14RukQ0ylM2GCdViUHBBjZ2imCaYcjlux/view?usp=sharing). Данные получены с акселерометров и гироскопов мобильных телефонов Samsung Galaxy S3 (подробнее о признаках по ссылке выше), также известен тип активности человека с телефоном в кармане — ходил ли он, стоял, лежал, сидел или поднимался/спускался по лестнице.

Сначала мы сделаем вид, что тип активности нам неизвестен, и попытаемся кластеризовать людей исключительно на основе имеющихся признаков. Затем решим задачу определения типа физической активности как задачу классификации.

In [30]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm_notebook

%matplotlib inline
from matplotlib import pyplot as plt
plt.style.use(['seaborn-v0_8-darkgrid'])
plt.rcParams['figure.figsize'] = (12, 9)
plt.rcParams['font.family'] = 'DejaVu Sans'

from sklearn import metrics
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

RANDOM_STATE = 17

In [31]:
# при необходимости измените путь
PATH_TO_SAMSUNG_DATA = "../data"

In [32]:
X_train = np.loadtxt(os.path.join(PATH_TO_SAMSUNG_DATA, "samsung_train.txt"))
y_train = np.loadtxt(os.path.join(PATH_TO_SAMSUNG_DATA,
                                  "samsung_train_labels.txt")).astype(int)

X_test = np.loadtxt(os.path.join(PATH_TO_SAMSUNG_DATA, "samsung_test.txt"))
y_test = np.loadtxt(os.path.join(PATH_TO_SAMSUNG_DATA,
                                  "samsung_test_labels.txt")).astype(int)

In [33]:
# Проверяем размерности
assert(X_train.shape == (7352, 561) and y_train.shape == (7352,))
assert(X_test.shape == (2947, 561) and y_test.shape == (2947,))

Для кластеризации вектор целевой переменной не нужен, поэтому будем работать с объединением обучающей и тестовой выборок. Объедините `X_train` с `X_test`, и `y_train` с `y_test`.

In [34]:
# ваш код здесь

Определите количество уникальных значений меток целевого класса.

In [35]:
# np.unique(y)

In [36]:
# n_classes = np.unique(y).size

[Эти метки соответствуют:](https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.names)
- 1 – ходьба
- 2 – подъём по лестнице
- 3 – спуск по лестнице
- 4 – сидение
- 5 – стояние
- 6 – лежание

Отмасштабируйте выборку с помощью `StandardScaler` с параметрами по умолчанию.

In [37]:
# ваш код здесь

Снизьте число измерений с помощью PCA, оставив столько компонент, сколько необходимо для объяснения как минимум 90% дисперсии исходных (масштабированных) данных. Используйте масштабированный датасет и зафиксируйте `random_state` (константа RANDOM_STATE).

In [38]:
# ваш код здесь
# pca = 
# X_pca = 

**Вопрос 1:**

Какое минимальное число главных компонент необходимо для покрытия 90% дисперсии исходных (масштабированных) данных?

In [39]:
# ваш код здесь

**Варианты ответа:**
- 56
- 65
- 66
- 193

**Вопрос 2:**

Какой процент дисперсии покрывается первой главной компонентой? Округлите до ближайшего процента.

**Варианты ответа:**
- 45
- 51
- 56
- 61

In [40]:
# ваш код здесь

Визуализируйте данные в проекции на первые две главные компоненты.

In [41]:
# ваш код здесь
# plt.scatter(, , c=y, s=20, cmap='viridis');

**Вопрос 3:**

Если всё сделано правильно, вы увидите несколько кластеров, почти идеально разделённых друг от друга. Какие типы активности входят в эти кластеры?

**Варианты ответа:**
- 1 кластер: все 6 активностей
- 2 кластера: (ходьба, подъём по лестнице, спуск по лестнице) и (сидение, стояние, лежание)
- 3 кластера: (ходьба), (подъём по лестнице, спуск по лестнице) и (сидение, стояние, лежание)
- 6 кластеров

---

Выполните кластеризацию методом `KMeans`, обучив модель на данных со сниженной размерностью (после PCA). В данном случае мы подскажем искать ровно 6 кластеров, но в общем случае мы не будем знать, сколько кластеров искать.

Параметры:

- **n_clusters** = n_classes (количество уникальных меток целевого класса)
- **n_init** = 100
- **random_state** = RANDOM_STATE (для воспроизводимости результата)

Остальные параметры оставьте по умолчанию.

In [42]:
# ваш код здесь

Визуализируйте данные в проекции на первые две главные компоненты. Раскрасьте точки в соответствии с полученными кластерами.

In [43]:
# ваш код здесь
# plt.scatter(, , c=cluster_labels, s=20, cmap='viridis');

Посмотрите на соответствие между метками кластеров и исходными метками классов, и на каких видах активности алгоритм `KMeans` ошибается.

In [44]:
# tab = pd.crosstab(y, cluster_labels, margins=True)
# tab.index = ['ходьба', 'подъём по лестнице',
#             'спуск по лестнице', 'сидение', 'стояние', 'лежание', 'всего']
# tab.columns = ['cluster' + str(i + 1) for i in range(6)] + ['всего']
# tab

Мы видим, что для каждого класса (т.е. каждой активности) есть несколько кластеров. Посмотрим на максимальную долю объектов одного класса, отнесённых к одному кластеру. Это будет простая метрика, характеризующая, насколько легко класс отделяется от других при кластеризации.

Пример: если для класса «спуск по лестнице» (1406 экземпляров) распределение по кластерам таково:
 - кластер 1 — 900
 - кластер 3 — 500
 - кластер 6 — 6,

то такая доля составит 900/1406 $\approx$ 0.64.

**Вопрос 4:**

Какая активность лучше всего отделяется от остальных по описанной выше метрике?

**Варианты ответа:**
- ходьба
- стояние
- спуск по лестнице
- все три варианта неверны

Видно, что KMeans не очень хорошо различает активности. Используйте метод «локтя» для выбора оптимального числа кластеров. Параметры алгоритма и используемые данные те же, что и раньше, меняем только `n_clusters`.

In [45]:
# # ваш код здесь
# inertia = []
# for k in tqdm_notebook(range(1, n_classes + 1)):
#     pass

**Вопрос 5:**

Сколько кластеров можно выбрать по методу «локтя»?

**Варианты ответа:**
- 1
- 2
- 3
- 4

---

Попробуем другой алгоритм кластеризации, описанный в статье — агломеративную кластеризацию.

In [46]:
# ag = AgglomerativeClustering(n_clusters=n_classes, 
#                              linkage='ward').fit(X_pca)

Рассчитайте Adjusted Rand Index (`sklearn.metrics`) для полученной кластеризации и для `KMeans` с параметрами из вопроса 4.

In [47]:
# ваш код здесь

**Вопрос 6:**

Выберите все верные утверждения.

**Варианты ответа:**
- По ARI, KMeans справился с кластеризацией хуже, чем агломеративная кластеризация
- Для ARI неважно, какие метки присвоены кластеру, важно лишь разбиение объектов на кластеры
- В случае случайного разбиения на кластеры ARI будет близок к нулю

---

Можно заметить, что задача решается не очень хорошо, когда мы пытаемся выделить несколько кластеров (> 2). Теперь решим задачу классификации, учитывая, что данные размечены.

Для классификации используем метод опорных векторов — класс `sklearn.svm.LinearSVC`. В рамках этого курса мы не изучали этот алгоритм отдельно, но он хорошо известен и можно почитать о нём, например, [здесь](http://cs231n.github.io/linear-classify/#svmvssoftmax).

Выберите гиперпараметр `C` для `LinearSVC` с помощью `GridSearchCV`.

- Обучите новый `StandardScaler` на обучающей выборке (со всеми исходными признаками), примените масштабирование к тестовой выборке
- В `GridSearchCV` укажите `cv` = 3.

In [48]:
# # ваш код здесь
# scaler = StandardScaler()
# X_train_scaled =
# X_test_scaled = 

In [49]:
svc = LinearSVC(random_state=RANDOM_STATE)
svc_params = {'C': [0.001, 0.01, 0.1, 1, 10]}

In [50]:
# %%time
# # ваш код здесь
# best_svc = None

In [51]:
# best_svc.best_params_, best_svc.best_score_

**Вопрос 7:**

Какое значение гиперпараметра `C` оказалось лучшим по результатам кросс-валидации?

**Варианты ответа:**
- 0.001
- 0.01
- 0.1
- 1
- 10

In [52]:
# y_predicted = best_svc.predict(X_test_scaled)

In [53]:
# tab = pd.crosstab(y_test, y_predicted, margins=True)
# tab.index = ['ходьба', 'подъём по лестнице',
#              'спуск по лестнице', 'сидение', 'стояние', 'лежание', 'всего']
# tab.columns = ['ходьба', 'подъём по лестнице',
#              'спуск по лестнице', 'сидение', 'стояние', 'лежание', 'всего']
# tab

**Вопрос 8:**

Какой тип активности хуже всего определяется SVM по precision? По recall?

**Варианты ответа:**
- precision — подъём по лестнице, recall — лежание
- precision — лежание, recall — сидение
- precision — ходьба, recall — ходьба
- precision — стояние, recall — сидение

Наконец, сделайте то же самое, что и в вопросе 7, но с применением PCA.

- Используйте `X_train_scaled` и `X_test_scaled`
- Обучите тот же PCA, что и раньше, на масштабированной обучающей выборке, примените преобразование к тестовой
- Выберите гиперпараметр `C` через кросс-валидацию на обучающей выборке с PCA-преобразованием. Вы заметите, насколько быстрее это работает.

**Вопрос 9:**

Какова разница между лучшим качеством (accuracy) по кросс-валидации при использовании всех 561 исходных признаков и при применении метода главных компонент? Округлите до ближайшего процента.

**Варианты ответа:**
- качество одинаковое
- 2%
- 4%
- 10%
- 20%

In [54]:
# ваш код здесь

**Вопрос 10:**

Выберите все верные утверждения:

**Варианты ответа:**
- Метод главных компонент в данном случае позволил сократить время обучения модели, при этом качество (средняя accuracy по кросс-валидации) сильно пострадало, более чем на 10%
- PCA можно использовать для визуализации данных, но для этой задачи есть лучшие методы, например t-SNE. Однако PCA имеет меньшую вычислительную сложность
- PCA строит линейные комбинации исходных признаков, и в некоторых приложениях они могут быть плохо интерпретируемы человеком